In [1]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder as sklearnOneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.validation import check_X_y, check_array, check_random_state
from sklearn.utils import column_or_1d
from sklearn.utils.validation import check_is_fitted

In [2]:
class MeanEncoder(BaseEstimator, TransformerMixin):
    """Encode categorical features using means of target feature.
    """
    def __init__(self):
        pass

    def fit(self, X, y):
        """Fit MeanEncoder
        Parameters
        ----------
        X : array-like of shape (n_samples,)
            Input array of any type
        y: array-like of shape (n_samples,)
            Target values for means calculation
        Returns
        -------
        self : returns an instance of self.
        """
        
        # Проверяем, содержится ли во входящих массивах Х и у 
        # хотя бы одно пустое значение
        # Если содержится, то выводим ошибку "Input contains NaN"
        if pd.Series(X).isnull().any():
            raise ValueError("Input contains NaN")

        if pd.Series(y).isnull().any():
            raise ValueError("Input contains NaN")
        
        # Проверяет входящие массивы Х и у на соответствие "длин"
        # Проверяет размерность у: является ли массив одномерным
        # Проверяет, больше ли двух размерность Х
        # Проверяет, входят ли в Х бесконечные или пустые значения
        # ensure_2d=False: в данном случае при одномерном Х не будет вызвана ошибка
        # dtype=None: в данном случае сохраняется dtype предикторов входящих массивов
        # copy=True: создаем копию входящих данных
        # Возвращает проверенные и конвертированные в numpy.ndarray X, y
        X, y = check_X_y(X, y, ensure_2d=False, dtype=None, copy=True)
        
        # Определяем количество уникальных значений в массиве Х
        # Записываем их в переменную classes_ экземпляра класса
        self.classes_ = np.unique(X)
        
        # Создаём список из массивов соответствий между значениями из Х
        # и значениями из self.classes_. Массивы состоят из True, False.
        # Сколько уникальных значений в self.classes_- столько массивов в indexes
        indexes = [(X == value) for value in self.classes_]
        
        # Для каждого массива из indexes считаем среднее значение массива по маске у:
        # True = соответсвующему значению из y. 
        # Записываем результат в means_ экземпляра класса
        self.means_ = [y[index].mean() for index in indexes]
        
        # Возвращаем экземпляр класса
        return self

    def fit_transform(self, X, y=None, **fit_params):
        """Fit MeanEncoder to X, then transform X.
        Equivalent to self.fit(X).transform(X)
        Parameters
        ----------
        X : array-like of shape (n_samples,)
            Input array of any type
        y: array-like of shape (n_samples,)
            Target values for means calculation
        Returns
        -------
        X* : array-like
        """
        # Проверяем, содержится ли во входящих массивах Х и у 
        # хотя бы одно пустое значение
        # Если содержится, то выводим ошибку "Input contains NaN"
        if pd.Series(X).isnull().any():
            raise ValueError("Input contains NaN")

        if pd.Series(y).isnull().any():
            raise ValueError("Input contains NaN")
            
        # Проверяет входящие массивы Х и у на соответствие "длин"
        # Проверяет размерность у: является ли массив одномерным
        # Проверяет, больше ли двух размерность Х
        # Проверяет, входят ли в Х бесконечные или пустые значения
        # ensure_2d=False: в данном случае при одномерном Х не будет вызвана ошибка
        # dtype=None: в данном случае сохраняется dtype предикторов входящих массивов
        # copy=True: создаем копию входящих данных
        # Возвращает проверенные и конвертированные в numpy.ndarray X, y
        X, y = check_X_y(X, y, ensure_2d=False, dtype=None, copy=True)
        
        # Определяем количество уникальных значений в массиве Х
        # Записываем их в переменную classes_ экземпляра класса        
        self.classes_ = np.unique(X)
        
        # Создаём список из массивов соответствий между значениями из Х
        # и значениями из self.classes_. Массивы состоят из True, False.
        # Сколько уникальных значений в self.classes_- столько массивов в indexes        
        indexes = [(X == value) for value in self.classes_]
        
        # Для каждого массива из indexes считаем среднее значение массива по маске у:
        # True = соответсвующему значению из y. 
        # Записываем результат в means_ экземпляра класса
        self.means_ = [y[index].mean() for index in indexes]
        
        # Присваиваем категориальному значению Х соответствующее ему среднее из self.means_.
        # Для этого создается объект-итератор, из которого при каждом обороте цикла извлекается кортеж, 
        # состоящий из двух элементов. Первый берется из списка indexes, второй - из self.means_.
        # Так как последовательность в indexes и self.means_ сохранена, выполняем присваивание
        for index, mean in zip(indexes, self.means_):
            X[index] = mean
            
        # Возвращаем преобразованный предиктор
        return X

    def transform(self, X):
        """ Transform X by fitted MeanEncoder.
        Parameters
        ----------
        X : array-like of shape (n_samples,)
        Input array of any type
        Returns
        -------
        X* : array-like
        """
        
        # Проверяем, был ли выполнен метод fit().
        # Для этого смотрим, существует ли атрибут classes_ у экземпляра класса
        check_is_fitted(self, 'classes_')
        
        # Проверяем, содержится ли во входящем массиве Х
        # хотя бы одно пустое значение
        # Если содержится, то выводим ошибку "Input contains NaN"        
        if pd.Series(X).isnull().any():
            raise ValueError("Input contains NaN")
            
        # Проверяет, больше ли двух размерность Х
        # Проверяет, входят ли в Х бесконечные или пустые значения
        # ensure_2d=False: в данном случае при одномерном Х не будет вызвана ошибка
        # dtype=None: в данном случае сохраняется dtype предикторов входящих массивов
        # copy=True: создаем копию входящих данных
        # Возвращает проверенные и конвертированные в numpy.ndarray X            
        X = check_array(X, ensure_2d=False, dtype=None, copy=True)

        # Определяем количество уникальных значений в массиве Х
        # Записываем их в переменную classes        
        classes = np.unique(X)
        
        # Проверяем, имеются ли в обучаемом массиве уникальные категориальные значения.
        # Для этого находим длину пересечения множеств classes и self.classes_.
        # Если она оказалась меньше длины множества classes, то выводим ошибку с расчётом,
        # сколько новых уровней содержит обучающий массив
        if len(np.intersect1d(classes, self.classes_)) < len(classes):
            diff = np.setdiff1d(classes, self.classes_)
            raise ValueError("X contains new labels: %s" % str(diff))
  
        # Создаём список из массивов соответствий между значениями из Х
        # и значениями из self.classes_. Массивы состоят из True, False.
        # Сколько уникальных значений в self.classes_- столько массивов в indexes  
        indexes = [(X == value) for value in self.classes_]
        
        # Присваиваем категориальному значению Х соответствующее ему среднее из self.means_.
        # Для этого создается объект-итератор, из которого при каждом обороте цикла извлекается кортеж, 
        # состоящий из двух элементов. Первый берется из списка indexes, второй - из self.means_.
        # Так как последовательность в indexes и self.means_ сохранена, выполняем присваивание
        for index, mean in zip(indexes, self.means_):
            X[index] = mean
        return X

In [25]:
# создаем игрушечный обучающий датафрейм pandas
toy_train = pd.DataFrame({'Balance': [8.3, 9.4, 10.2, 0],
                          'Age': [23, 29, 36, 44],
                          'Male': ["M", "M", "W", "M"]})

In [51]:
X, y = check_X_y(toy_train["Male"], toy_train["Age"], ensure_2d=False, dtype=None, copy=True)

In [52]:
X.dtype


dtype('O')

In [53]:
X

array(['M', 'M', 'W', 'M'], dtype=object)

In [54]:
y

array([23, 29, 36, 44], dtype=int64)

In [55]:
type(y)

numpy.ndarray

In [56]:
X == "M"

array([ True,  True, False,  True])

In [57]:
classes_ = np.unique(X)
classes_

array(['M', 'W'], dtype=object)

In [58]:
indexes = [(X == value) for value in classes_]

indexes

[array([ True,  True, False,  True]), array([False, False,  True, False])]

In [59]:
means_ = [y[index].mean() for index in indexes]

means_

[32.0, 36.0]

In [63]:
for index in indexes:
    print(index)

[ True  True False  True]
[False False  True False]
